In [ ]:
!git clone https://github.com/ZhaoJ9014/face-aging-CAAE.git


Cloning into 'face-aging-CAAE'...
remote: Enumerating objects: 424, done.
remote: Total 424 (delta 0), reused 0 (delta 0), pack-reused 424 (from 1)
Receiving objects: 100% (424/424), 195.22 MiB | 23.14 MiB/s, done.
Resolving deltas: 100% (229/229), done.
Updating files: 100% (29/29), done.


In [ ]:
# Install 2to3 tool if not available
!pip install -q 2to3
# Convert the code from Python 2 to Python 3
!2to3 -W -n -f all -o py3_converted face-aging-CAAE


lib2to3.main: Output in 'py3_converted' will mirror the input directory 'face-aging-CAAE' layout.
RefactoringTool: Skipping optional fixer: buffer
RefactoringTool: Skipping optional fixer: idioms
RefactoringTool: Skipping optional fixer: set_literal
RefactoringTool: Skipping optional fixer: ws_comma
RefactoringTool: Refactored face-aging-CAAE/FaceAging.py
--- face-aging-CAAE/FaceAging.py	(original)
+++ face-aging-CAAE/FaceAging.py	(refactored)
@@ -7,7 +7,7 @@
 # Please cite above paper if you use this code
 #
 
-from __future__ import division
+
 import os
 import time
 from glob import glob
@@ -73,7 +73,7 @@
             name='z_prior'
         )
         # ************************************* build the graph *******************************************************
-        print '\n\tBuilding graph ...'
+        print('\n\tBuilding graph ...')
 
         # encoder: input image --> z
         self.z = self.encoder(
@@ -408,16 +408,16 @@
                     }
                 )
 
-   

In [ ]:
!pip install scipy Pillow opencv-python


In [ ]:
%cd py3_converted


/content/py3_converted


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving test.jpg to test.jpg


In [ ]:
import os
import shutil

os.makedirs("test_images", exist_ok=True)
for fname in uploaded.keys():
    shutil.move(fname, os.path.join("test_images", fname))


In [ ]:
file_path = "/content/py3_converted/ops.py"

with open(file_path, "r") as file:
    code = file.read()

code = code.replace(
    "from scipy.misc import imread, imresize, imsave",
    "from PIL import Image\nimport numpy as np"
)

# Replace imread, imresize, imsave usage
code = code.replace("imread(", "np.array(Image.open(")
code = code.replace("imresize(", "np.array(Image.open(")
code = code.replace("imsave(", "Image.fromarray(")

# Patch any .astype('uint8') after resizing
code = code.replace(".astype('uint8')", ")")

with open(file_path, "w") as file:
    file.write(code)

print("✅ Patched deprecated scipy.misc calls in ops.py")


✅ Patched deprecated scipy.misc calls in ops.py


In [ ]:
!head -n 100 /content/py3_converted/ops.py | tail -n 20


    elif len(x_shape) == 4:
        label = tf.reshape(label, [x_shape[0], 1, 1, label_shape[-1]])
        return tf.concat(axis=3, values=[x, label*tf.ones([x_shape[0], x_shape[1], x_shape[2], label_shape[-1]])])


def load_image(
        image_path,  # path of a image
        image_size=64,  # expected size of the image
        image_value_range=(-1, 1),  # expected pixel value range of the image
        is_gray=False,  # gray scale or color image
):
    if is_gray:
        image = np.array(Image.open(image_path, mode='L').astype(np.float32)
    else:
        image = np.array(Image.open(image_path, mode='RGB').astype(np.float32)
    image = np.array(Image.open(image, [image_size, image_size])
    image = image.astype(np.float32) * (image_value_range[-1] - image_value_range[0]) / 255.0 + image_value_range[0]
    return image




In [ ]:
file_path = "/content/py3_converted/ops.py"

with open(file_path, "r") as f:
    lines = f.readlines()

# Print around the problematic area
for i, line in enumerate(lines[85:105], start=85):
    print(f"{i:>3}: {line.rstrip()}")


 85: def load_image(
 86:         image_path,  # path of a image
 87:         image_size=64,  # expected size of the image
 88:         image_value_range=(-1, 1),  # expected pixel value range of the image
 89:         is_gray=False,  # gray scale or color image
 90: ):
 91:     if is_gray:
 92:         image = np.array(Image.open(image_path, mode='L').astype(np.float32)
 93:     else:
 94:         image = np.array(Image.open(image_path, mode='RGB').astype(np.float32)
 95:     image = np.array(Image.open(image, [image_size, image_size])
 96:     image = image.astype(np.float32) * (image_value_range[-1] - image_value_range[0]) / 255.0 + image_value_range[0]
 97:     return image
 98: 
 99: 
100: def save_batch_images(
101:         batch_images,   # a batch of images
102:         save_path,  # path to save the images
103:         image_value_range=(-1,1),   # value range of the input batch images
104:         size_frame=None     # size of the image matrix, number of images in each row an

In [ ]:
correct_code = """
def load_image(
        image_path,  # path of a image
        image_size=64,  # expected size of the image
        image_value_range=(-1, 1),  # expected pixel value range of the image
        is_gray=False,  # gray scale or color image
):
    if is_gray:
        image = Image.open(image_path).convert('L')
    else:
        image = Image.open(image_path).convert('RGB')

    image = image.resize((image_size, image_size))
    image = np.array(image).astype(np.float32)
    image = image * (image_value_range[-1] - image_value_range[0]) / 255.0 + image_value_range[0]
    return image
"""

file_path = "/content/py3_converted/ops.py"

with open(file_path, "r") as f:
    lines = f.readlines()

# Replace old load_image block
start = 85
end = 98
lines[start:end+1] = correct_code.strip().split('\n')

# Save updated file
with open(file_path, "w") as f:
    f.write('\n'.join(lines))

print("✅ Fixed `load_image()` function in ops.py")


✅ Fixed `load_image()` function in ops.py


In [ ]:
%tensorflow_version 1.x


ValueError: Tensorflow 1 is unsupported in Colab.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

In [ ]:
!grep -rn 'restore' /content/py3_converted/


grep: /content/py3_converted/__pycache__/FaceAging.cpython-311.pyc: binary file matches
/content/py3_converted/old_version/FaceAging.py:620:            self.saver.restore(self.session, os.path.join(checkpoint_dir, checkpoints_name))
/content/py3_converted/FaceAging.py:647:                self.saver.restore(self.session, os.path.join(checkpoint_dir, checkpoints_name))


In [ ]:
!find . -name "*.ckpt*"


In [ ]:
!find /content/face-aging-CAAE -name "main.py"


/content/face-aging-CAAE/main.py
/content/face-aging-CAAE/old_version/main.py


In [ ]:
!2to3 -W -n -f all /content/face-aging-CAAE -o /content/face-aging-CAAE_py3


lib2to3.main: Output in '/content/face-aging-CAAE_py3' will mirror the input directory '/content/face-aging-CAAE' layout.
RefactoringTool: Skipping optional fixer: buffer
RefactoringTool: Skipping optional fixer: idioms
RefactoringTool: Skipping optional fixer: set_literal
RefactoringTool: Skipping optional fixer: ws_comma
RefactoringTool: Refactored /content/face-aging-CAAE/FaceAging.py
--- /content/face-aging-CAAE/FaceAging.py	(original)
+++ /content/face-aging-CAAE/FaceAging.py	(refactored)
@@ -7,7 +7,7 @@
 # Please cite above paper if you use this code
 #
 
-from __future__ import division
+
 import os
 import time
 from glob import glob
@@ -73,7 +73,7 @@
             name='z_prior'
         )
         # ************************************* build the graph *******************************************************
-        print '\n\tBuilding graph ...'
+        print('\n\tBuilding graph ...')
 
         # encoder: input image --> z
         self.z = self.encoder(
@@ -408,16 +408,16 

In [ ]:
!python3 /content/face-aging-CAAE_py3/main.py --is_train False --testdir /content/face-aging-CAAE_py3/test_images --savedir /content/face-aging-CAAE_py3/results


2025-04-10 07:48:01.987597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744271282.025512   25599 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744271282.036928   25599 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Instructions for updating:
non-resource variables are not supported in the long term
2025-04-10 07:48:07.045853: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Namespace(is_train=True, epoch=50, dataset='UTKFace', savedir='save', testdir='None', use_trained_model=True, use_init_model=True)

	Building graph ...

	Training Mode
I0

In [ ]:
!python3 /content/face-aging-CAAE_py3/main.py --is_train=False --testdir=/content/face-aging-CAAE_py3/test_images --savedir=/content/face-aging-CAAE_py3/results --use_trained_model=True --use_init_model=True


2025-04-10 08:29:16.364827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744273756.392585   35487 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744273756.400872   35487 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Instructions for updating:
non-resource variables are not supported in the long term
2025-04-10 08:29:23.265675: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Namespace(is_train=True, epoch=50, dataset='UTKFace', savedir='save', testdir='None', use_trained_model=True, use_init_model=True)

	Building graph ...

	Training Mode
I0

In [ ]:
!cat /content/face-aging-CAAE/init_model/model_parts/* > /content/face-aging-CAAE/init_model/model-init.data-00000-of-00001


cat: /content/face-aging-CAAE/init_model/model_parts/save: Is a directory


In [ ]:
!mkdir -p /content/face-aging-CAAE_py3/init_model/model_parts


In [ ]:
%cd /content/face-aging-CAAE/init_model/model_parts


/content/face-aging-CAAE/init_model/model_parts


In [ ]:
!ls -lh /content/face-aging-CAAE/init_model/


total 180M
-rw-r--r-- 1 root root   36 Apr 10 06:04 checkpoint
-rw-r--r-- 1 root root    2 Apr 10 06:04 __init__.py
-rw-r--r-- 1 root root 179M Apr 10 07:16 model-index.ckpt
-rw-r--r-- 1 root root    0 Apr 10 08:35 model-init.data-00000-of-00001
-rw-r--r-- 1 root root 2.3K Apr 10 06:04 model-init.index
-rw-r--r-- 1 root root 657K Apr 10 06:04 model-init.meta
drwxr-xr-x 3 root root 4.0K Apr 10 08:35 model_parts
-rw-r--r-- 1 root root 2.0K Apr 10 06:04 zip_opt.py


In [ ]:
!ls -lh /content/face-aging-CAAE/init_model/model_parts/


total 179M
-rw-r--r-- 1 root root    0 Apr 10 08:36 model-init.data-00000-of-00001
-rw-r--r-- 1 root root  24M Apr 10 06:04 part0001
-rw-r--r-- 1 root root  24M Apr 10 06:04 part0002
-rw-r--r-- 1 root root  24M Apr 10 06:04 part0003
-rw-r--r-- 1 root root  24M Apr 10 06:04 part0004
-rw-r--r-- 1 root root  24M Apr 10 06:04 part0005
-rw-r--r-- 1 root root  24M Apr 10 06:04 part0006
-rw-r--r-- 1 root root  24M Apr 10 06:04 part0007
-rw-r--r-- 1 root root  15M Apr 10 06:04 part0008
drwxr-xr-x 3 root root 4.0K Apr 10 07:48 save


In [ ]:
!python3 /content/face-aging-CAAE_py3/main.py --is_train=False --testdir=/content/face-aging-CAAE_py3/test_images --savedir=/content/face-aging-CAAE_py3/results --use_trained_model=True --use_init_model=True


2025-04-10 08:33:23.911674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744274003.935851   36505 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744274003.942829   36505 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Instructions for updating:
non-resource variables are not supported in the long term
2025-04-10 08:33:29.471947: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Namespace(is_train=True, epoch=50, dataset='UTKFace', savedir='save', testdir='None', use_trained_model=True, use_init_model=True)

	Building graph ...

	Training Mode
I0

In [ ]:
%cd /content/face-aging-CAAE/init_model/model_parts


/content/face-aging-CAAE/init_model/model_parts


In [ ]:
!file part0001


part0001: data


In [ ]:
!cat /content/face-aging-CAAE/init_model/model_parts/part000* > /content/face-aging-CAAE/init_model/model_combined


In [ ]:
# Try unzip
!unzip /content/face-aging-CAAE/init_model/model_combined -d /content/face-aging-CAAE/init_model/unpacked_model


Archive:  /content/face-aging-CAAE/init_model/model_combined
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/face-aging-CAAE/init_model/model_combined or
        /content/face-aging-CAAE/init_model/model_combined.zip, and cannot find /content/face-aging-CAAE/init_model/model_combined.ZIP, period.


In [ ]:
# Try tar
!tar -xvf /content/face-aging-CAAE/init_model/model_combined -C /content/face-aging-CAAE/init_model/unpacked_model


tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [ ]:
!mv /content/face-aging-CAAE/init_model/model_combined /content/face-aging-CAAE/init_model/model-index.ckpt
